# Transformation des variables text en num

**Ce code est destiné à transformer les réponses de l'utilisateur qui peuvent être à la fois numérique mais aussi textuelles en variables numériques.**

Voici les variables que nous récupérons des réponses de l'utilisateur : 
- Nom
- Prénom
- Adresse
- Distance du festival
- Date
- Catégorie de festival
- Sous-catégorie
- Taille du festival souhaité
- Budget
- Durée du festival

## 1. Adresse

**Le premier objectif est de transformer l'adresse postale en coordonnées géographqiues**

Le code est récupéré du site officiel de data.gouv pour réaliser la requête API : https://guides.data.gouv.fr/reutiliser-des-donnees/utiliser-les-api-geographiques/utiliser-lapi-adresse/geocoder-des-adresses-pratique.
Cela nous permet de transformer l'adresse postale au format : Numéro de rue Type de rue Nom de la rue Ville Pays Code postal (5 Avenue Le Chatelier Palaiseau France 91120)

In [ ]:
import requests
ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'
params = {
    'q': '24 Rue des Diables Bleus 73000 Chambéry',
    'limit': 5
}
response = requests.get(ADDOK_URL, params=params)
j = response.json()
if len(j.get('features')) > 0:
    first_result = j.get('features')[0]
    lon, lat = first_result.get('geometry').get('coordinates')
    first_result_all_infos = { **first_result.get('properties'), **{"lon": lon, "lat": lat}}
    print(first_result_all_infos)
else:
    print('No result')

{'label': '24 Rue des diables bleus 73000 Chambéry', 'score': 0.9657545454545454, 'housenumber': '24', 'id': '73065_1110_00024', 'name': '24 Rue des diables bleus', 'postcode': '73000', 'citycode': '73065', 'x': 927213.22, 'y': 6501342.72, 'city': 'Chambéry', 'context': '73, Savoie, Auvergne-Rhône-Alpes', 'type': 'housenumber', 'importance': 0.6233, 'street': 'Rue des diables bleus', 'lon': 5.91394, 'lat': 45.573853}


On améliore le code pour qu'il corresponde plus à nos demandes, à savoir retourner uniquement les coordonnées géographiques. On gère aussi les erreurs possibles et on utilise ce code comme une fonction.

In [2]:
import requests

def adresse_to_coordgeo(adresse):
    """Convertit une adresse en coordonnées géographiques en utilisant l'API de DataGouv."""
    ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'
    params = {
        'q': adresse,
        'limit': 1  # On ne veut que le premier résultat
    }
    
    try:
        response = requests.get(ADDOK_URL, params=params)
        response.raise_for_status()  # Vérifie que la requête a réussi
        j = response.json()
        
        # Vérifier si des résultats ont été trouvés
        if len(j.get('features')) > 0:
            first_result = j.get('features')[0]
            lon, lat = first_result.get('geometry').get('coordinates')
            first_result_all_infos = {**first_result.get('properties'), **{"lon": lon, "lat": lat}}
            return (lat, lon)
        else:
            print('No result')
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'appel à l'API : {e}")
        return None

In [3]:
adresse_to_coordgeo('5 Avenue Le Chatelier Palaiseau France 91120')

(48.711413, 2.207679)

On vérifie pour 10 adresses que les coordonnés géographiques sont justes : 

In [ ]:
Adresses = {
    "24 Rue des Diables Bleus, 73000 Chambéry, France": (45.574069, 5.914342),
    "5 Avenue Victor Hugo, 75016 Paris, France": (48.868942, 2.283629),
    "10 Place de la Bourse, 33000 Bordeaux, France": (44.841354, -0.570416),
    "17 Rue Saint-Nicolas, 54000 Nancy, France": (48.688457, 6.186303),
    "12 Rue du Château, 44000 Nantes, France": (47.216293, -1.551646),
    "8 Rue de la Poulaillerie, 69002 Lyon, France": (45.764228, 4.833735),
    "7 Rue de la Liberté, 21000 Dijon, France": (47.322166, 5.037889),
    "15 Boulevard de la Croisette, 06407 Cannes, France": (43.551420, 7.021624),
    "3 Rue St-Rome, 31000 Toulouse, France": (43.602706, 1.443534),
    "11 Avenue Jean Médecin, 06000 Nice, France": (43.698753, 7.269191)
}

In [ ]:
def arrondir_coordonnees(coord, precision=2):
    """Arrondir une coordonnée à une certaine précision (décimales)"""
    return round(coord, precision)

for adresse in Adresses.keys():
    # Obtenir les coordonnées réelles
    lat_connue, lon_connue = Adresses[adresse]
    
    # Obtenir les coordonnées via la fonction adresse_to_coordgeo()
    coords = adresse_to_coordgeo(adresse)
    
    if coords:
        lat_obtenue, lon_obtenue = coords

        # Arrondir les deux ensembles de coordonnées
        lat_connue_arr = arrondir_coordonnees(lat_connue)
        lon_connue_arr = arrondir_coordonnees(lon_connue)
        lat_obtenue_arr = arrondir_coordonnees(lat_obtenue)
        lon_obtenue_arr = arrondir_coordonnees(lon_obtenue)
        
        # Vérifier si les coordonnées correspondent après arrondi
        if (lat_connue_arr, lon_connue_arr) != (lat_obtenue_arr, lon_obtenue_arr):
            print(f"Faux pour : {adresse}")
        else:
            print(f"Correct pour : {adresse}")
    else:
        print(f"Aucune coordonnée trouvée pour : {adresse}")

Correct pour : 24 Rue des Diables Bleus, 73000 Chambéry, France
Faux pour : 5 Avenue Victor Hugo, 75016 Paris, France
Correct pour : 10 Place de la Bourse, 33000 Bordeaux, France
Faux pour : 17 Rue Saint-Nicolas, 54000 Nancy, France
Correct pour : 12 Rue du Château, 44000 Nantes, France
Correct pour : 8 Rue de la Poulaillerie, 69002 Lyon, France
Correct pour : 7 Rue de la Liberté, 21000 Dijon, France
Correct pour : 15 Boulevard de la Croisette, 06407 Cannes, France
Correct pour : 3 Rue St-Rome, 31000 Toulouse, France
Correct pour : 11 Avenue Jean Médecin, 06000 Nice, France


On remarque que pour certaines adresses, on retrouve des différences. Heureusement pour nous, ces différences sont seulement au centième près. Cela représente une distance de 1km environ, ce qui ne pose pas énormément de problèmes dans notre cas. Je pense même que le problème vient des données du dictionnaire Adresses. Les données que je vais récupérer ne sont pas totalement justes.

**Le deuxième objectif est de calculer la distance entre 2 points, qui sont l'adresse de l'utilisateur et l'adresse du festival**

Pour ce faire, nous allons utiliser la formule : **Distance de haversine**.

$$
d = 2R \cdot \arcsin\left(\sqrt{\sin^2\left(\frac{\Delta \text{lat}}{2}\right) + \cos(\text{lat}_1) \cdot \cos(\text{lat}_2) \cdot \sin^2\left(\frac{\Delta \text{lon}}{2}\right)}\right)
$$

$$
\begin{aligned}
d & : \text{ distance entre les deux points (en mètres)} \\
R & : \text{ rayon de la Terre} \\
\text{lat}_1, \text{lon}_1 & : \text{ latitude et longitude du premier point} \\
\text{lat}_2, \text{lon}_2 & : \text{ latitude et longitude du second point} \\
\Delta \text{lat} & = \text{lat}_2 - \text{lat}_1 : \text{ différence de latitude} \\
\Delta \text{lon} & = \text{lon}_2 - \text{lon}_1 : \text{ différence de longitude}
\end{aligned}
$$



In [ ]:
from math import *
def distance_haversine(loc1, loc2):
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    R = (6378137 + 6356752)/2 #On prend la moyenne entre le rayon équatorial et le rayon polaire
    d = 2 * R * atan2(sqrt(sin(delta_lat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(delta_lon / 2)**2), 
                      sqrt(1 - (sin(delta_lat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(delta_lon / 2)**2)))

    return d

La distance entre Paris et Lyon est de 391280.45 mètres.


On connait désormais la distance entre 2 points. Le problème est que c'est la distance à vol d'oiseau. On reste pour le moment sur cette méthode pour vérifier si la festival est assez proche de l'adresse de la personne.

On implémente un nouveau code pour vérifier si le festival est à une distance inférieure à une distance limite.

In [ ]:
def distance_valid(distance, limit):
    if distance <= limit :
        return True
    return False


distance_valid(50, 40)

False